In [1]:
import sys
import os

# Get the absolute path of the parent directory.
parent_dir = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

# Add the parent directory to the system path to be able to import modules from 'lib.'
sys.path.append(parent_dir)

In [2]:
import gensim.downloader as api

from lib.memory import DSDM
from lib.utils import cleanup, configs, inference, learning, preprocess, utils

import numpy as np
import pandas as pd
import pickle
import plotly.express as px
from sklearn.manifold import TSNE

[nltk_data] Downloading package punkt to
[nltk_data]     /nfs/home/dfichiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /nfs/home/dfichiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load cleanup and memory
cleanup = pickle.load(open("cleanups/cleanup_2023-08-13 15:39:34.086703.pkl",'rb'))
memory = pickle.load(open("memories/memory_2023-08-13 15:39:34.086703.pkl",'rb'))

In [4]:
%%capture

# Load pre-trained word embeddings (Word2Vec in this example)
word_vectors = api.load("word2vec-google-news-300")

In [8]:
%%capture
address_embeddings = []
address_concepts = []

for address in memory.addresses[1000:2000]:
    tokens = inference.get_most_similar_HVs(inference.get_similarities_to_atomic_set(address, cleanup))
    embeddings = [word_vectors[word] for word in tokens if word in word_vectors]
    if embeddings:
        address_concepts.append(" ".join(tokens))
        address_embeddings.append(sum(embeddings) / len(embeddings))

In [9]:
reduced_embeddings = TSNE(n_components=2, random_state=42, perplexity=2).fit_transform(np.array(address_embeddings))

df = pd.DataFrame(reduced_embeddings, columns=["Dimension 1", "Dimension 2"])
df["Chunk"] = address_concepts

In [10]:
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Chunk")
fig.show()

In [ ]:
def remove_duplicates(addresses):
    for 